In [ ]:
# import models
import pandas as pd
import numpy as np

#pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 

#models
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

**IMPORT DATA**

In [ ]:
from project_fraud.data import cleaned_featured_data

In [ ]:
df = cleaned_featured_data('~/data/')

In [ ]:
df.head()

**SPLIT DATA**

In [ ]:
X = df[['TransactionID','P_emaildomain_bin','card1','card2','addr1','TransactionAmt','card5','D15','C13','D2','D10','D4','weekday','hours','dist_mean', 'dist_median','dist_mean_rel','dist_median_rel']]
y = df['isFraud']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.00017, random_state=1)

**SAVE VAL_DATA**

In [ ]:
X_val.to_csv('val_data.csv', index =False, na_rep='nan')

**PIPELINES**

In [ ]:
n = (X.dtypes != 'object')
num_cols = list(n[n].index)
medium_missing_num_cols = []
low_missing_num_cols =[]
for i in num_cols:
    percentage = df[i].isnull().sum() * 100 / len(df[i])
    if percentage < 15:
        low_missing_num_cols.append(i)
    elif percentage >= 15 and percentage <= 60:
        medium_missing_num_cols.append(i)

In [ ]:
# trainer.py

num_transformer_low = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])
num_transformer_medium = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('one_hot', OneHotEncoder())
])
    
preprocessor = ColumnTransformer([
    ('low_num_imputer',num_transformer_low, low_missing_num_cols),
    ('medium_num_imputer', num_transformer_medium, medium_missing_num_cols),
    ('cat_transformer', cat_pipeline, ['P_emaildomain_bin','weekday','hours'])],
    remainder='drop')

**MODEL SVC**

In [ ]:
pipeline_svc = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", SVC()),
     ])

In [ ]:
pipeline_svc.fit(X_train, y_train)

In [ ]:
pipeline_svc.predict(X_val)

In [ ]:
pipeline_svc.predict_proba(X_val)